# Efficientnet baseline by Pytorch

* model : efficientnet-b0(pretrained)
* loss : CategoricalCrossEntropy(+label smoothing)
* optimizer : Adam
* preprocessing
 * RandomResizedCrop
 * RandomHorizontalFlip
 * RandomVerticalFlip
 * ColorJitter
 * ToTensor
 * Normalize
*  TTA : True


# log

ver2 : add "RondomResizeCrop" and "ColorJitter"  
ver3 : reduce the effect of "ColorJitter", add labelsmoothing  
ver4 : add TTA  
ver5 : add TTA3, increace epoch(13->25)  
ver6 : add cutmix, add fmix  
ver7 : delete fmix, change smoothing(0.2 -> 0.1)  
ver8 : delete rotate, change bbx size, change epoch(25->20)  
ver9 : imsize 512->600, change smoothing(0.1 -> 0.2), change ColorJitter rate  
ver10: using alubmentation(Fundamentally changed the augmentation)

## Library import

In [ ]:
import numpy as np
import pandas as pd
import json
from PIL import Image
import os
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from albumentations import HorizontalFlip, VerticalFlip, ShiftScaleRotate, Transpose, HueSaturationValue, RandomResizedCrop, RandomBrightnessContrast, Cutout, Compose, Normalize, CoarseDropout
from albumentations.pytorch import ToTensorV2
import cv2

import sys
import random

# add efficientnet path
efnet_path = '../input/efficientnet-pytorch/EfficientNet-PyTorch/EfficientNet-PyTorch-master'
sys.path.append(efnet_path)
from efficientnet_pytorch import EfficientNet

# add fmix path
fmix_path = '../input/image-fmix/FMix-master' #'../input/efficientnet-pytorch-07/efficientnet_pytorch-0.7.0'
sys.path.append(fmix_path)
from fmix import sample_mask


## Directory Setting

In [ ]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

OUTPUT_DIR = './'
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

TRAIN_DIR = '../input/cassava-leaf-disease-classification/train_images/'
TEST_DIR = '../input/cassava-leaf-disease-classification/test_images/'

## Data loading

In [ ]:
labels = json.load(open("../input/cassava-leaf-disease-classification/label_num_to_disease_map.json"))
train = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
sample = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')

X, Y = train['image_id'].values, train['label'].values
X_test = [name for name in (os.listdir(TEST_DIR))]

## Config

In [ ]:
class Conf:
    seed=777
    
    BATCH = 16
    EPOCHS = 20

    LR = 0.0001
    IM_SIZE = 600

    n_fold=5
    target_col='label'
    
    modelname='efficientnet-b0'

## Helper Functions

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(Conf.seed)

def get_img(path):
    im_bgr = cv2.imread(path)
    im_rgb = im_bgr[:, :, ::-1]
    return im_rgb

## CV split

In [ ]:
folds = train.copy()
Fold = StratifiedKFold(n_splits=Conf.n_fold, shuffle=True, random_state=Conf.seed)
for n, (train_index, val_index) in enumerate(Fold.split(folds, folds[Conf.target_col])):
    folds.loc[val_index, 'fold'] = int(n)
folds['fold'] = folds['fold'].astype(int)
print(folds.groupby(['fold', Conf.target_col]).size())

## Augmentation

In [ ]:
def train_transform():
    return Compose([
            RandomResizedCrop(Conf.IM_SIZE, Conf.IM_SIZE),
            Transpose(p=0.5),
            HorizontalFlip(0.5),
            VerticalFlip(p=0.5),
            ShiftScaleRotate(0.5),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1,0.1), p=0.5),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
            CoarseDropout(p=0.5),
            ToTensorV2(p=1.0)
])

def test_transform():
    return Compose([
            RandomResizedCrop(Conf.IM_SIZE, Conf.IM_SIZE, (1.0,1.0), ratio=(1.0,1.0)),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1.0),
            ToTensorV2(p=1.0)
])

def TTA1():
    return Compose([
            RandomResizedCrop(Conf.IM_SIZE, Conf.IM_SIZE),
            HorizontalFlip(1.0),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1.0),
            ToTensorV2(p=1.0)
])

def TTA2():
    return Compose([
            RandomResizedCrop(Conf.IM_SIZE, Conf.IM_SIZE),
            VerticalFlip(1.0),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1.0),
            ToTensorV2(p=1.0)
])

def TTA3():
    return Compose([
            RandomResizedCrop(Conf.IM_SIZE, Conf.IM_SIZE),
            HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=1.0),
            RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1,0.1), p=1.0),
            Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], p=1.0),
            ToTensorV2(p=1.0)
])

TTAs = [test_transform, TTA1, TTA2, TTA3]

## Dataset/Dataloader

In [ ]:
class TrainData(Dataset):
    def __init__(self, Dir, FNames, Labels, Transform):
        self.dir = Dir
        self.fnames = FNames
        self.transform = Transform
        self.lbs = Labels
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):
        img = get_img(os.path.join(self.dir, self.fnames[index]))  
        img = self.transform()(image=img)['image']
        return img, self.lbs[index] 
        
class TestData(Dataset):
    def __init__(self, Dir, FNames, TTAs=TTAs):
        self.dir = Dir
        self.fnames = FNames
#         self.transform = Transform
        self.ttas = TTAs
        
    def __len__(self):
        return len(self.fnames)

    def __getitem__(self, index):
        img = get_img(os.path.join(self.dir, self.fnames[index]))
        imglist = [tta()(image=img)['image'] for tta in self.ttas]
        imglist = torch.stack(imglist)
        return imglist, self.fnames[index]

In [ ]:
trn_idx = folds[folds['fold'] != 0].index
val_idx = folds[folds['fold'] == 0].index

X_train, Y_train = X[trn_idx], Y[trn_idx]
X_val, Y_val = X[val_idx], Y[val_idx]

trainset = TrainData(TRAIN_DIR, X_train, Y_train, train_transform)
trainloader = DataLoader(trainset,
                         batch_size=Conf.BATCH,
                         shuffle=True,
                         num_workers=4)

validset = TrainData(TRAIN_DIR, X_val, Y_val, test_transform)
validloader = DataLoader(validset,
                         batch_size=Conf.BATCH,
                         shuffle=False,
                         num_workers=4)

testset = TestData(TEST_DIR, X_test, TTAs)
testloader = DataLoader(testset,
                        batch_size=1,
                        shuffle=False,
                        num_workers=4)

### Cutmix and Fmix

In [ ]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)
    return bbx1, bby1, bbx2, bby2

def cutmix(data, target, alpha):
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_target = target[indices]

    lam = np.clip(np.random.beta(alpha, alpha),0.5,0.6)
    bbx1, bby1, bbx2, bby2 = rand_bbox(data.size(), lam)
    new_data = data.clone()
    new_data[:, :, bby1:bby2, bbx1:bbx2] = data[indices, :, bby1:bby2, bbx1:bbx2]
    # adjust lambda to exactly match pixel ratio
    lam = 1 - ((bbx2 - bbx1) * (bby2 - bby1) / (data.size()[-1] * data.size()[-2]))
    targets = (target, shuffled_target, lam)

    return new_data, targets

def fmix(data, targets, alpha, decay_power, shape, max_soft=0.0, reformulate=False):
    lam, mask = sample_mask(alpha, decay_power, shape, max_soft, reformulate)
    mask =torch.tensor(mask, device=DEVICE).float()
    indices = torch.randperm(data.size(0))
    shuffled_data = data[indices]
    shuffled_targets = targets[indices]
    x1 = mask.to(DEVICE)*data
    x2 = (1-mask).to(DEVICE)*shuffled_data
    targets=(targets, shuffled_targets, lam)
    
    return (x1+x2), targets

### Image Visualization

In [ ]:
class UnNormalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, images):

        images = images.mul_(self.std).add_(self.mean)
            # The normalize code -> t.sub_(m).div_(s)
        return images

mean = torch.tensor([0.485, 0.456, 0.406]).view(1,3,1,1)
std = torch.tensor([0.229, 0.224, 0.225]).view(1,3,1,1)
inv_normalize = UnNormalize(mean, std)

In [ ]:
row = 4
col = 4
fig, axes = plt.subplots(row, col, figsize=(16, 16))
axes = axes.ravel()

images,labels = next(iter(trainloader))
images,_ = cutmix(images, labels, 1.)
images = inv_normalize(images)
images = images.detach().numpy().transpose(0,2,3,1)

for j in range(Conf.BATCH):
    axes[j].imshow(images[j])

## Model

In [ ]:
# Model
class enetv2(nn.Module):
    def __init__(self, out_dim=1, ModelName="efficientnet-b0"):
        super(enetv2, self).__init__()
        self.basemodel = EfficientNet.from_name(Conf.modelname)
        
        self.myfc = nn.Linear(self.basemodel._fc.in_features, out_dim)
        self.basemodel._fc = nn.Identity()        
            
    def extract(self, x):
        return self.basemodel(x)

    def forward(self, x):
        x = self.basemodel(x)
        x = self.myfc(x)
        return x
    
# Loss
class LabelSmoothingLoss(nn.Module):

    def __init__(self, classes, smoothing=0.0, dim=-1): 
        super(LabelSmoothingLoss, self).__init__() 
        self.confidence = 1.0 - smoothing 
        self.smoothing = smoothing 
        self.cls = classes 
        self.dim = dim 
    def forward(self, pred, target): 
        pred = pred.log_softmax(dim=self.dim) 
        with torch.no_grad(): 
            true_dist = torch.zeros_like(pred) 
            true_dist.fill_(self.smoothing / (self.cls - 1)) 
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence) 
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [ ]:
model = enetv2(5, Conf.modelname)
checkpoint = torch.load("../input/efficientnet-pytorch/efficientnet-b0-08094119.pth", map_location=DEVICE)
model.load_state_dict(checkpoint, strict=False)
model = model.to(DEVICE)

criterion = LabelSmoothingLoss(5, 0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=Conf.LR)
scheduler = ReduceLROnPlateau(optimizer, mode='min', verbose=True)

# Training

In [ ]:
def train_one_epoch(model, train_loader):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    
    for batch_idx, (images, labels) in enumerate(train_loader):
        images = images.to(DEVICE)
        labels = labels.to(DEVICE)
        
        #cutmix and fmix
#         mix_decision = np.random.rand()
#         if mix_decision < 0.25:
#             images, labels = cutmix(images, labels, 1.)
#         elif mix_decision >= 0.25 and mix_decision < 0.5:
#             images, labels = fmix(images, labels, alpha=1, decay_power=5., shape=(512,512))

        mix_decision = np.random.rand()
        if mix_decision < 0.5:
            images, labels = cutmix(images, labels, 1.)
        
        optimizer.zero_grad() 
        outputs = model(images)
        
        if mix_decision < 0.5:
            loss = criterion(outputs, labels[0]) * labels[2] + criterion(outputs, labels[1]) * (1. - labels[2])
            labels = labels[0]
        else:
            loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predict = torch.max(outputs.data, 1)
        correct += (predict == labels).sum().item()
        total += labels.size(0)
        
    train_loss = running_loss / len(train_loader)
    train_acc = correct / total
    
    return train_loss, train_acc

def valid_one_epoch(model, valid_loader):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0
    
    with torch.no_grad():
        
        for batch_idx, (images, labels) in enumerate(valid_loader):
            images = images.to(DEVICE)
            labels = labels.to(DEVICE)
            
            outputs = model(images)
            
            loss = criterion(outputs, labels)
            running_loss += loss.item()
            
            _, predict = torch.max(outputs.data, 1)
            correct += (predict == labels).sum().item()
            total += labels.size(0)
            
    val_loss = running_loss / len(valid_loader)
    val_acc = correct / total
    
    return val_loss, val_acc

#### For saving quota

In [ ]:
# best_score = 0.

# if len(testloader) >= 2:
#     for epoch_idx in range(Conf.EPOCHS):

#         train_loss, train_acc = train_one_epoch(model, trainloader)
#         valid_loss, valid_acc = valid_one_epoch(model, validloader)

#         # model save
#         if valid_acc > best_score:
#             best_score = valid_acc

#             torch.save(model.state_dict(), OUTPUT_DIR+f'{Conf.modelname}_best.pth')

#             print('model saved')

#         # rl scheduler
#         scheduler.step(valid_loss)

#         print('Epoch: {} |train_loss: {:.3f} valid loss: {:.3f} train_acc: {:.3f} valid_acc: {:.3f}'.format(epoch_idx, train_loss, valid_loss, train_acc, valid_acc))

In [ ]:
best_score = 0.

for epoch_idx in range(Conf.EPOCHS):

    train_loss, train_acc = train_one_epoch(model, trainloader)
    valid_loss, valid_acc = valid_one_epoch(model, validloader)

    # model save
    if valid_acc > best_score:
        best_score = valid_acc

        torch.save(model.state_dict(), OUTPUT_DIR+f'{Conf.modelname}_best.pth')

        print('model saved')

    # rl scheduler
    scheduler.step(valid_loss)

    print('Epoch: {} |train_loss: {:.3f} valid loss: {:.3f} train_acc: {:.3f} valid_acc: {:.3f}'.format(epoch_idx, train_loss, valid_loss, train_acc, valid_acc))

## Inference

In [ ]:
if len(testloader) >= 2:
    model.load_state_dict(torch.load(OUTPUT_DIR+f'{Conf.modelname}_best.pth'), strict=False)
s_ls = []

with torch.no_grad():
    model.eval()
    for images, fname in testloader: 
        images = images.to(DEVICE)
        batch_size, n_crops, c, h, w = images.size()
        images = images.view(-1, c, h, w)
        output = model(images)
        output = output.mean(0)
        ps = torch.exp(output)
        _, top_class = ps.topk(1, dim=0)
        
        s_ls.append([fname[0], top_class.item()])

## Make submission

In [ ]:
sub = pd.DataFrame.from_records(s_ls, columns=['image_id', 'label'])
sub.to_csv("submission.csv", index=False)

In [ ]:
sub.head()